<BR>
<BR>
<BR>
<BR>
<BR>
<BR>
   
                                          
                                          
                                          
                                          
                                          
#                                             Bagged Model in Predicting Obesity Risk




##                                                        Gladys Murage

##                              College of Business, Engineering, and  Technology, National University

##                                         DDS8555 v1: PREDICTIVE ANALYSIS(3602869492)

##                                                        Dr MOHAMED NABEEL

##                                                            April 15, 2025


<BR>
<BR>
<BR>
<BR>
<BR>
<BR>
<BR>
<BR>

# Key Components Explained:
### Base Estimator:
The model uses the same Decision Tree parameters from the previous model

### Bagging Parameters:
1. n_estimators=50. Creates 50 decision trees
2. max_samples=0.8. Each tree uses 80% random sample of training data
3. max_features=0.8. Each tree uses 80% random features

### Advantages Over Single Decision Tree:
1. Reduces variance (overfitting)
2. Typically provides better generalization
3. Handles class imbalance better through random sampling

### Output Includes:
1. Validation accuracy and full classification report
2. Feature importance analysis (averaged across all trees)
3. Test predictions in submission format

In [3]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

# Load data
train = pd.read_csv('Otrain.csv')
test = pd.read_csv('Otest.csv')
sample_sub = pd.read_csv('Osample_submission.csv')

# Separate features and target
X = train.drop('NObeyesdad', axis=1)
y = train['NObeyesdad']

# Preprocessing: Convert categorical variables to numerical
ordinal_encoders = {}
for column in X.select_dtypes(include=['object']).columns:
    oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    X[column] = oe.fit_transform(X[[column]]).flatten()
    ordinal_encoders[column] = oe

# Encode target variable if categorical
if y.dtype == 'object':
    le_target = LabelEncoder()
    y = le_target.fit_transform(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create Bagging classifier
bagging_model = BaggingClassifier(
    estimator=DecisionTreeClassifier(
        max_depth=5,
        min_samples_split=10,
        random_state=42
    ),
    n_estimators=50,
    max_samples=0.8,
    max_features=0.8,
    random_state=42,
    n_jobs=-1
)

# Train the model
bagging_model.fit(X_train, y_train)

# Make predictions on validation set
y_pred = bagging_model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

# Prepare test set predictions
if not test.empty:
    # Apply the same preprocessing to test data
    for column in test.select_dtypes(include=['object']).columns:
        if column in ordinal_encoders:
            test[column] = ordinal_encoders[column].transform(test[[column]]).flatten()
    
    # Make predictions
    test_pred = bagging_model.predict(test)
    
    # If we encoded the target, inverse transform to get original labels
    if 'le_target' in locals():
        test_pred = le_target.inverse_transform(test_pred)
    
    # Create submission file
    submission = sample_sub.copy()
    submission['NObeyesdad'] = test_pred
    submission.to_csv('bagging_submission.csv', index=False)
    print("\nTest predictions saved to 'bagging_submission.csv'")

Validation Accuracy: 0.8572

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.92      0.92       524
           1       0.84      0.80      0.82       626
           2       0.86      0.77      0.81       543
           3       0.97      0.96      0.96       657
           4       1.00      1.00      1.00       804
           5       0.63      0.78      0.70       484
           6       0.72      0.68      0.70       514

    accuracy                           0.86      4152
   macro avg       0.85      0.84      0.84      4152
weighted avg       0.86      0.86      0.86      4152


Test predictions saved to 'bagging_submission.csv'


# Interpretation of the Bagging Model
## Validation Accuracy: 0.8572
Performance Improvement. This indicates a better model accuracy compared to the decision tree model's previous result (83.69%). It's a positive step forward, suggesting that the adjustments made by ensemble techniques, like bagging, are working effectively. The weighted F1-score of 0.86 confirms good overall performance across all classes.

## Detailed Classification Analysis
### Class 0
Precision and  Recall both remain at 0.92, showing consistent high performance. F1-Score of 0.92 reflects a perfect balance between precision and recall. Class 0 remains a strong performer, with no significant concerns.

### Class 1:
Precision shows 84% accuracy in predictions for Class 1. Recall is improved to 80%, indicating better identification of true instances within class 1. F1-Scoreof 0.82 now balances precision and recall more effectively. The higher recall suggests reduced false negatives, which is a positive step for this class.

### Class 2:
Precision is 86%, which is slightly higher than before in the decision tree. Recall remains at 77%, showing room for improvement in true positive identification. The bagging model only identifies correct instances 77% of the time and 33% of the time gets it wrong. F1-Score of 0.81 demonstrates solid overall performance. Class 2 continues to perform well, though a focus on recall optimization might further enhance results.

### Class 3:
Precision and  Recall are both near perfection at 0.97 and 0.96, respectively. Meaning that bagging gets the class classified accurately 97% of the time and gets the instances correct 96% of the time. An F1-Scoreof 0.96 remains stellar. Class 3 is evidently well-separated and easy to classify.

### Class 4:
Precision, Recall, and  F1-Score all remain at 1.00, reflecting flawless performance. This class benefits from highly distinct features, requiring little to no further adjustments.

### Class 5:
Precision is improved to 63%, showing better prediction accuracy but the model is still mis-classifying 37% of this class. Recall remains high at 78%, reflecting strong identification of true instances when the model classifies. F1-Score now at 0.70, indicates improved balance. While precision has improved, further work is still needed to address false positives in this class.

### Class 6:
Precision, Recall, and F1-Score metrics are consistent with the previous results, hovering around 0.70. Class 6 remains steady but could benefit from focused tuning to enhance both precision and recall.

## Macro and Weighted Averages
Macro Average (0.85). Shows improved overall balance between precision and recall across all classes. Weighted Average (0.86). Slightly higher than before in the decision tree, reflecting better performance for larger classes like 4 and 3.

## Insights on Bagging
Bagging is an ensemble technique and unlike decision trees, bagging helps reduce overfitting and variance, which may explain the improved accuracy and precision for several classes. However, Class 5 and Class 6 still seem to struggle comparatively, likely due to overlapping features or insufficient representation in the dataset.

## Key Observations
### Class Imbalance Impact:
1. Larger classes (3,4) perform better than smaller ones (5,6). Weighted avg (0.86) > macro avg (0.84) confirms this

### Bagging Benefits:
1. 2% accuracy boost over single Decision Tree
2. More balanced recall across classes
3. Better handling of Class 1 (recall improved from 0.65 to 0.80)

### Persistent Challenges:
1. Class 5 still has precision issues with many false positives.
2. Class 6 remains problematic with both precision and recall < 0.75.

## Recommendations for Improvement
1. For class 5's low precision increase weight
2. For class 6's general weakness consider feature engineering for distinguishing characteristics
3. For the entire bagging model, consider trying random forest for better performance.
4. Advanced techniques such as hyper-parameter tuning and Grid search may help improve results

## Business Implications
### Reliable Predictions:
The bagging model can confidently predict Classes 0, 3, and 4. Use cases requiring these predictions are production-ready

### Caution Areas:
Applications relying on Class 5 or 6 predictions need human verification. Consider collecting more data for these classes

## Next Steps:
The 85.7% accuracy may be sufficient for many applications.,For critical use cases, consider stacking with other models

This model represents a significant improvement over the initial Decision Tree, particularly in handling class imbalance and improving recall for problematic classes. The remaining challenges with Classes 5 and 6 are likely due to either:
1. Insufficient distinguishing features in the data
2. Genuine ambiguity in these categories
3. Need for more sophisticated modeling techniques